作者：绿洲元
链接：https://www.zhihu.com/question/275901829/answer/1574659804
来源：知乎
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

写了一个python脚本下载全部表情包

访问API获取表情包信息
在登录的状态下访问 http://api.bilibili.com/x/emote/setting/panel?business=reply ，返回的json另存为 emoji.json

运行脚本

In [1]:
import json
import os
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from time import sleep

import requests
from tqdm import tqdm


def mkdir(path):
    if os.path.exists(path) is not True:
        os.makedirs(path)


def download_pic(url, path):
    resp = requests.get(url)
    with open(path, 'wb') as code:
        code.write(resp.content)


def download_emoji(emoji, save_path):
    emoji_name = emoji['text']
    emoji_name = emoji_name[1:-1]
    path = os.path.join(save_path, f'{emoji_name}.png')
    download_pic(emoji['url'], path)


def download_package(package, path):
    package_name = package['text']
    save_path = os.path.join(path, package_name)
    mkdir(save_path)
    all_emote = set(map(lambda e: e['text'][1:-1], package['emote']))
    downloaded_emote = set(map(lambda e: e[:-4], os.listdir(save_path)))
    download_emote_name = all_emote - downloaded_emote
    if len(download_emote_name) == 0:
        return
    going_to_download_emote = filter(lambda e: e['text'][1:-1] in download_emote_name, package['emote'])
    with ThreadPoolExecutor() as executor:
        executor.map(partial(download_emoji, save_path=save_path), going_to_download_emote)
    sleep(3)


In [3]:
with open('emoji.json', 'r', encoding='utf-8') as fp:
    emojis = json.load(fp)
emoji_packages = emojis['data']['all_packages']
package_path = 'bilibili_emoji'
bar = tqdm(emoji_packages)
for p in bar:
    if p['text'] == '颜文字':
        continue
    bar.set_description(p['text'])
    download_package(p, package_path)

坎公骑冠剑: 100%|█████████████████████████████████████████████████████████████████████████| 205/205 [11:05<00:00,  3.15s/it]


In [2]:
with open('emoji.json', 'r', encoding='utf-8') as fp:
    emojis = json.load(fp)
emoji_packages = emojis['data']['all_packages']
package_path = 'bilibili_emoji'
bar = tqdm(emoji_packages)


  0%|                                                                                          | 0/205 [00:00<?, ?it/s]

In [8]:
for p in bar:
    if p['text'] == '颜文字':
        continue
#     bar.set_description(p['text'])
    
    package=p
    path=package_path
    package_name = package['text']
    save_path = os.path.join(path, package_name)
    mkdir(save_path)
    all_emote = set(map(lambda e: e['text'][1:-1], package['emote']))
    downloaded_emote = set(map(lambda e: e[:-4], os.listdir(save_path)))
    download_emote_name = all_emote - downloaded_emote
    going_to_download_emote = filter(lambda e: e['text'][1:-1] in all_emote, package['emote'])
    print(going_to_download_emote)
#     with ThreadPoolExecutor() as executor:
#         executor.map(partial(download_emoji, save_path=save_path), going_to_download_emote)
#     sleep(3)
    break
